In [74]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import json
import pickle

In [ ]:
Rating:
Archive Warning:
#check for category (optional)
Category:
#check for fandom
Fandom:
#check for relationship (optional)
Relationship:
#check for character
Character:
#check for additional tags (optional)
Additional Tags:
#check for series (optional)
Series:
#check for collections (optional)
Collections:
#check for stats
Stats:
    


In [75]:
soup_0005 = BeautifulSoup(open('Fanfic_HTML/0005.html'), "html.parser")
soup_13Heirs = BeautifulSoup(open('Fanfic_HTML/13 Heirs.html'), "html.parser")
soup_WizardComes = BeautifulSoup(open('Fanfic_HTML/In Which A Wizard Comes.html'), "html.parser")

In [78]:
def storyElements(soup_obj):
    local_classTags = soup_obj.find(class_="tags")
    i = 0
    for element in local_classTags:
        print(i)
        print(element)
        i += 1

In [79]:
storyElements(soup_0005)
storyElements(soup_13Heirs)
storyElements(soup_WizardComes)

0


1
<dt>Rating:</dt>
2


3
<dd><a href="http://archiveofourown.org/tags/General%20Audiences">General Audiences</a></dd>
4


5
<dt>Archive Warning:</dt>
6


7
<dd><a href="http://archiveofourown.org/tags/Choose%20Not%20To%20Use%20Archive%20Warnings">Choose Not To Use Archive Warnings</a></dd>
8


9
<dt>Category:</dt>
10


11
<dd><a href="http://archiveofourown.org/tags/Gen">Gen</a></dd>
12


13
<dt>Fandom:</dt>
14


15
<dd><a href="http://archiveofourown.org/tags/The%20Umbrella%20Academy%20(TV)">The Umbrella Academy (TV)</a>, <a href="http://archiveofourown.org/tags/The%20Umbrella%20Academy%20(Comics)">The Umbrella Academy (Comics)</a></dd>
16


17
<dt>Character:</dt>
18


19
<dd><a href="http://archiveofourown.org/tags/Number%20Five%20%7C%20The%20Boy%20(Umbrella%20Academy)">Number Five | The Boy (Umbrella Academy)</a></dd>
20


21
<dt>Additional Tags:</dt>
22


23
<dd><a href="http://archiveofourown.org/tags/Eating%20Disorders">Eating Disorders</a>, <a href="http://archiveofourown.or

In [127]:
def extractStoryRating(classTags):
    #code to extract story rating
    i = 0
    for element in classTags:
        i += 1
        if i == 4:
            story_rating = str(element.find('a').string)
        elif i > 4:
            break
            
    return story_rating

def extractArchiveWarning(classTags):
    i = 0
    for element in classTags:
        i += 1
        if i == 8:
            archiveWarning = str(element.find('a').string)
        elif i > 8:
            break
            
    return archiveWarning

def extractCategory(classTags):
    #code to extract category
    i = 0
    for element in classTags:
        i += 1
        if i == 12:
            story_category = str(element.find('a').string)
        elif i > 12:
            break

    return story_category

def extractFandoms(classTags):
    #code to extract fandoms
    i = 0
    j = 0
    fandom_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Fandom:":
            j = 2 + i
        elif i == j:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += str(obj.string) + ', '
            fandom_text = fandom_text[:-2]

    return fandom_text

def extractAddTags(classTags):
    #code to extract tags
    i = 0
    j = 0
    additionalTags_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Additional Tags:":
            j = 2 + i
        elif i == j:
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += str(obj.string) + ', '
            additionalTags_text = additionalTags_text[:-2]

    return additionalTags_text

In [128]:
classTags = soup_13Heirs.find(class_="tags")
dict_relationships = extractRelationships(classTags)

In [129]:
dict_relationships

'Sophie Devereaux/Nathan Ford, Alec Hardison/Parker/Eliot Spencer'

In [130]:
def extractTags(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    fandom_text = ""
    story_character = np.nan
    addTags = np.nan
    archiveWarning = np.nan
    category = np.nan
    fandoms = np.nan
    pubDate = np.nan
    wordCount = np.nan
    tagsClass = soup_obj.find(class_="tags")
    for element in tagsClass:
        if str(element.string) == "Rating:":
            rating = extractStoryRating(tagsClass)
            
        elif str(element.string) == "Archive Warning:":
            archiveWarning = extractArchiveWarning(tagsClass)
            
        elif str(element.string) == "Category:":
            category = extractCategory(tagsClass)
            
        elif str(element.string) == "Fandom:":
            #check if fandom or relationship
            fandom_text = extractFandoms(tagsClass)
            
        elif i == 19:
            #check if fandom or character
            story_character = element.string
            
        elif str(element.string) == "Additional Tags:":
            addTags = extractAddTags(tagsClass)
            
        elif str(element.string) == "Stats:":
            pub_stat = str(element.string).strip()
            pubDate = pub_stat[11:21]
            wordCount = pub_stat[37:40]
            
        i += 1
        
    return story_character, fandom_text, addTags, rating, archiveWarning, category, pubDate, wordCount #, story_series_name, story_collections

In [131]:
story_character, fandom_text, additionalTags, story_rating, story_archive_warning, story_category, story_publish_date, story_word_count = extractTags(soup_13Heirs)

In [132]:
print(story_character)
print(fandom_text)
print(additionalTags) 
print(story_rating)
print(story_archive_warning)
print(story_category)
print(story_publish_date) 
print(story_word_count)

None
Sophie Devereaux/Nathan Ford, Alec Hardison/Parker/Eliot Spencer
Leverage
Future Fic, Post-Canon, Kid Fic, Found Family, Community: 14valentines
General Audiences
No Archive Warnings Apply
Multi




In [49]:
def extractAuthor(soup_obj):
    #code to extract author and fandom info
    for info in soup_obj.title:
        author = str(info).split('-')[1]
        author = author.strip()
    return author
    

def extractTitleStory(soup_obj):
    #code to extract title and story text
    i = 0
    story_text = ""
    for element in soup_obj.find(id="chapters"):
        if i == 1:
            title = element.string
        elif i == 3:
            story_ptag_text = element.find_all('p')
            for obj in story_ptag_text:
                if obj.string == None:
                    continue
                else:
                    story_text += str(obj.string) + " "
        i += 1
    return title, story_text
        
def extractTags(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    fandom_text = ""
    additionalTags_text = ""
    rating = np.nan
    archiveWarning = np.nan
    category = np.nan
    fandoms = np.nan
    pubDate = np.nan
    wordCount = np.nan
    for element in soup_obj.find(class_="tags"):
        if i == 3:
            story_rating = element.string
        elif i == 7:
            story_archive_warning = element.string
        elif i == 11:
            story_category = element.string
        elif i == 15:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += obj.string + ", "
            fandom_text = fandom_text[:-2]
        elif i == 19:
            story_character = element.string
            print(element)
        elif i == 23:
            print(element)
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += obj.string + ", "
                additionalTags_text = additionalTags_text[:-2]
        elif soup_obj.find(text="Stats:") == "Stats:":
            pub_stat = str(element.string).strip()
            story_publish_date = pub_stat[11:21]
            story_word_count = pub_stat[37:40]
        i += 1
   
    return story_character, fandom_text, additionalTags_text, story_rating, story_archive_warning, story_category, story_publish_date, story_word_count #, story_series_name, story_collections

def extractStoryURL(soup_obj): 
    #code to extract story URL
    i = 0
    for element in soup_obj.find(class_="message"):
        i += 1
        if i == 7:
            url = element['href']

        else:
            continue
    return url

def createAO3dict(filelist):
    i = 0
    ao3_dict = {}
    for element in filelist:
        soup = BeautifulSoup(open(element), "html.parser")
        url = extractStoryURL(soup)
        story_text = extractTitleStory(soup)
        character, fandom_text, additionalTags_text, rating, archive_warning, category, pub_date, word_count = extractTags(soup) #, story_series_name, story_collections
        author = extractAuthor(soup)
        title, story_text = extractTitleStory(soup)
        
        story_dict = {'title': title,
                      'author': author,
                      'story_text': story_text, 
                      'character': character,
                      'relationships' : relationships,
                      'url': url,
                      'fandom' : fandom_text,
                      'additionalTags': additionalTags_text, 
                      'rating' : rating, 
                      'archiveWarnings': archive_warning, 
                      'category' : category, 
                      'pub_date' : pub_date, 
                      'word_count' : word_count
                     }
        ao3_dict[i] = story_dict
        i += 1
    return ao3_dict

In [24]:
fanfic = ['Fanfic_HTML/13 Heirs.html']
fanfics = ['Fanfic_HTML/13 Heirs.html', 'Fanfic_HTML/In Which A Wizard Comes.html', 'Fanfic_HTML/Sweet Basil and Italian.html', 
           'Fanfic_HTML/5 6 7 The Umbrella.html', 'Fanfic_HTML/In Which A Wizard Is.html', 'Fanfic_HTML/Swine and Forests.html',
           'Fanfic_HTML/50 Truths of the.html', 'Fanfic_HTML/In Which Calcifer Meets.html', 'Fanfic_HTML/Symbol of Reform.html',
           'Fanfic_HTML/A Bakers Proposal.html', 'Fanfic_HTML/In Which Howl Attracts.html', 'Fanfic_HTML/Sympathy.html']

In [25]:
ao3_dictionary = createAO3dict(fanfic)

<dd><a href="http://archiveofourown.org/tags/Sophie%20Devereaux*s*Nathan%20Ford">Sophie Devereaux/Nathan Ford</a>, <a href="http://archiveofourown.org/tags/Alec%20Hardison*s*Parker*s*Eliot%20Spencer">Alec Hardison/Parker/Eliot Spencer</a></dd>
<dd><a href="http://archiveofourown.org/tags/Sophie%20Devereaux%20(Leverage)">Sophie Devereaux (Leverage)</a>, <a href="http://archiveofourown.org/tags/Parker%20(Leverage)">Parker (Leverage)</a></dd>


In [15]:
ao3_df = pd.DataFrame.from_dict(ao3_dictionary, orient="index")
ao3_df

title            author  \
0                                            13 Heirs         mayachain   
1                  In Which A Wizard Comes Home Drunk     bobblemonkey2   
2                     Sweet Basil and Italian Parsley       jenna_thorn   
3                        5 6 7 | The Umbrella Academy    AthenaPantheon   
4       In Which A Wizard Is Injured At A Rugby Match     bobblemonkey2   
5                                   Swine and Forests     Sour_Idealist   
6   50 Truths of the Umbrella Academy, by Allyson ...  GunpowderGelatin   
7                 In Which Calcifer Meets His Destiny         Untherius   
8                                    Symbol of Reform     cherryfeather   
9                                  A Baker's Proposal           hanluvr   
10  In Which Howl Attracts Customers and Sophie Pl...     bobblemonkey2   
11                                           Sympathy             Tibby   

                                           story_text  \
0   Sophie will never admit to the relief she feel...   
1   A clatter of pots and pans awoke Sophie in the...   
2   The thing is, Parker has no respect for the li...   
3   When her brothers were alive, Seven wasn’t alo...   
4   ‘Owwwww!’ Howl stomped in through the front do...   
5   “There’s a lot of magic around here, certainly...   
6   50 Truths of the Umbrella Academy - By Allyson...   
7   Calcifer floated in a state of weightlessness....   
8   He sets down his bottle with the other empties...   
9   I flee to the woods that have always been my r...   
10  Howl had been quite glum ever since the Witch ...   
11  Hunter rarely slept, but she knew it was essen...   

                             character  \
0                                 None   
1                                 None   
2   Alec Hardison/Parker/Eliot Spencer   
3                                 None   
4         Sophie Hatter/Howl Pendragon   
5         Sophie Hatter/Howl Pendragon   
6                                 None   
7         Sophie Hatter/Howl Pendragon   
8                                 None   
9       Katniss Everdeen/Peeta Mellark   
10        Sophie Hatter/Howl Pendragon   
11                         Hunter/Door   

                                          url  \
0    http://archiveofourown.org/works/3284888   
1   http://archiveofourown.org/works/24759217   
2    http://archiveofourown.org/works/6747331   
3   http://archiveofourown.org/works/25230484   
4   http://archiveofourown.org/works/24504826   
5     http://archiveofourown.org/works/222260   
6   http://archiveofourown.org/works/18002909   
7     http://archiveofourown.org/works/301901   
8     http://archiveofourown.org/works/407223   
9    http://archiveofourown.org/works/2657978   
10  http://archiveofourown.org/works/25864924   
11     http://archiveofourown.org/works/48750   

                                               fandom  \
0                                            Leverage   
1   Howl’s Moving Castle, Howl no Ugoku Shiro | Ho...   
2                                            Leverage   
3   Ben Hargreeves & Klaus Hargreeves, Number Five...   
4   Howl's Moving Castle - All Media Types, Howl S...   
5   Howl Series - Diana Wynne Jones, Enchanted For...   
6   The Umbrella Academy (TV), The Umbrella Academ...   
7                     Howl Series - Diana Wynne Jones   
8              Hunger Games Trilogy - Suzanne Collins   
9   Hunger Games Trilogy - Suzanne Collins, The Hu...   
10  Howl no Ugoku Shiro | Howl's Moving Castle, Ho...   
11                                Neverwhere - Gaiman   

                                       additionalTags                 rating  \
0      Sophie Devereaux (Leverage), Parker (Leverage)      General Audiences   
1   Calcifer (Howl Series), Howl Pendragon, Sophie...      General Audiences   
2                                 OT3, Domestic Fluff  Teen And Up Audiences   
3                                                                  Not Rated   
4  